In [1]:
import pandas as pd
import scipy.stats as stats
from datetime import datetime , timedelta
from scipy.stats import percentileofscore

In [2]:
filepath = "oldexpiries (1).csv"
dataset = pd.read_csv(filepath)

In [3]:
def get_next_date_from_csv(filepath, input_date, idx):
   dataset['expdt'] = pd.to_datetime(dataset['expdt']).dt.date
   
   dataset['expdt'] = pd.to_datetime(dataset['expdt'])
   filtered_df = dataset[dataset['idx'] == idx]
  
   greater_dates = filtered_df[filtered_df['expdt'] > input_date]
   if greater_dates.empty:
    print("No greater dates found.")
    return None
    
   next_date = greater_dates['expdt'].min()
    
   return next_date

def calculate_DTE(start_date, start_time, expiry_date):
    # Combine start date and time into a datetime object
    start_datetime = datetime.strptime(start_date + ' ' + start_time, '%Y-%m-%d %H:%M:%S')
    
    # Set expiry time to 15:30:00
    expiry_time = '15:30:00'
    
    # Combine expiry date and time into a datetime object
    expiry_datetime = datetime.strptime(expiry_date.strftime('%Y-%m-%d') + ' ' + expiry_time, '%Y-%m-%d %H:%M:%S')
    # Calculate the difference in minutes between start datetime and expiry datetime
    delta_minutes = (expiry_datetime - start_datetime).total_seconds() / 60
    
    return delta_minutes

In [4]:
file_path = "sorted_dataset.csv"
data = pd.read_csv(file_path)

In [5]:

print(data.head(5))


    DTE   CE   PE  Total
0  1441   66   63    129
1  1441   73   60    133
2  1441   70   81    151
3  1441   78  108    186
4  1441  129  119    248


In [6]:

grouped = data.groupby('DTE').agg(
    avg_CE=('CE', 'mean'),
    std_CE=('CE', 'std')
).reset_index()


def calculate_percentile(dte, value, option_type):
    dte_data = data[data['DTE'] == dte]
    
    if dte_data.empty:
        print(f"No data found for DTE: {dte}")
        return

    if option_type == 'CE':
        available_values = dte_data['CE']
    elif option_type == 'PE':
        available_values = dte_data['PE']
    elif option_type == 'Total':
        available_values = dte_data['Total']
    else:
        print("Invalid option type. Choose 'CE', 'PE', or 'Total'.")
        return

    average_value = available_values.mean()
    pricing = "cheap"
    case1 = "unusually"
    if value > average_value:
        pricing = "expensive"

    nearest_value = available_values.iloc[(available_values - value).abs().argsort()[:1]].values[0]
    percentile_score = percentileofscore(available_values, nearest_value)
    if 30 < percentile_score < 65:
        case1 = "normally"

    print(f"Average {option_type}: {average_value}")
    print(f"Percentile of {option_type} value {value}: {percentile_score}%")
    print(f"Option is {case1} {pricing}")


input_date =  input("Enter the date (yyyy-mm-dd): ")
time_str =  input("Enter the time (HH:MM:SS): ")
index = input("Enter the index (e.g., NIFTY or BANKNIFTY): ")


next_date = get_next_date_from_csv(file_path, input_date, index)

if next_date is not None:
    date_value = next_date.date()
    DTE = calculate_DTE(input_date, time_str, date_value)
    print(f"The expiry date {input_date} for index {index} is: {date_value}")
    print(f"The DTE value: {DTE}")
    
    option_type1 = "CE" 
    option_value1 = float(input("Enter the CE Value: ")) 
    calculate_percentile(DTE, option_value1, option_type1)

    option_type2 = "PE" 
    option_value2 = float(input("Enter the PE value: ")) 
    calculate_percentile(DTE, option_value2, option_type2)

    option_type3 = "Total"
    option_value3 = option_value1 + option_value2 
    calculate_percentile(DTE, option_value3, option_type3)
else:
    print("No next date found.")



The expiry date 2022-01-07 for index NIFTY is: 2022-01-13
The DTE value: 8820.0
Average CE: 167.94736842105263
Percentile of CE value 70.0: 2.6315789473684212%
Option is unusually cheap
Average PE: 158.71052631578948
Percentile of PE value 80.0: 2.6315789473684212%
Option is unusually cheap
Average Total: 326.6578947368421
Percentile of Total value 150.0: 2.6315789473684212%
Option is unusually cheap
